In [ ]:
#-- Install ultralytics for YOLO  --------------------------------------------------------------------------------
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Import Libraries ---------------------------------------------------------------------------------
import os
import glob
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import random
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Load Model -----------------------------------------------------------------------------------
model = YOLO('yolo11x.pt')
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Process video and save frame numbers with human -------------------------------------------
def process_video(video_file, output_dir):   
   
    
    video_name = os.path.basename(video_file).split('.')[0]
    output_file = os.path.join(output_dir, f"{video_name}_human.npy")

    # Open the video file
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Error opening video file: {video_file}")
        return
        

    frame_list_with_humans = []
    frame_number = 0

    while True:
        ret, frame = cap.read()  
        if not ret:
            break     

        #-- Run YOLO model inference on the frame --
        results = model.predict(frame, conf=0.5, verbose=False)

        #-- Check if any person is detected in the frame --
        human_found = False
        for result in results:
            for box in result.boxes:
                if box.cls == 0:  #-- Class 0 is typically 'person' in YOLO models --
                    frame_list_with_humans.append(frame_number)
                    human_found = True
                    break  #-- Break to avoid duplicate addition --
                
            if human_found:
                break
            
        frame_number += 1

    #-- Release the video capture object --
    cap.release()

    #-- Save the frame numbers as a NumPy array --
    np.save(output_file, np.array(frame_list_with_humans))
    print(f"Processed {video_name}: {len(frame_list_with_humans)} frames with humans saved to {output_file}")

    return output_file
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Check and Display frames include humans --------------------------------------------------
def display_random_detected_frames(video_path, frames_file, num_frames_to_show=10):
    #-- Load the saved frame numbers --
    detected_frames = np.load(frames_file)
    print(f"Loaded {len(detected_frames)} frames from {frames_file}")

    #-- If there are fewer frames than the number requested, adjust automatically --
    if len(detected_frames) < num_frames_to_show:
        num_frames_to_show = len(detected_frames)

    #-- Select random frames to display --
    random_frames = random.sample(list(detected_frames), num_frames_to_show)
    print(f"Randomly selected frames to display: {random_frames}")
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    for frame_number in random_frames:
        #-- Set the video to the specific frame --
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)  #-- Frame indices are 0-based

        ret, frame = cap.read()
        if not ret:
            print(f"Could not read frame {frame_number}")
            continue

        #-- Convert the frame from BGR to RGB for matplotlib --
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        #-- Display the frame using matplotlib --
        plt.figure(figsize=(10, 6))
        plt.imshow(frame_rgb)
        plt.title(f"Frame {frame_number}")
        plt.axis('off')
        plt.show()

    
    cap.release()
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Run on Burglary Samples ----------------------------------------------------------------------------
burglary_samples_dir = "/kaggle/input/novin-create-binary-burglary-ds/burglary_samples/"  
burglary_output_dir = "burglary_samples"  

os.makedirs(burglary_output_dir, exist_ok=True)

video_files = glob.glob(os.path.join(burglary_samples_dir, "*.mp4"))  
print(f'Number of Videos: {len(video_files)}')

for video_file in video_files:
    frames_file = process_video(video_file, burglary_output_dir)
    # display_random_detected_frames(video_file, frames_file, num_frames_to_show=5)
    # break
#---------------------------------------------------------------------------------------------------------------  

In [ ]:
#-- Run on Not Burglary Samples ----------------------------------------------------------------------------
not_burglary_samples_dir = "/kaggle/input/novin-create-binary-burglary-ds/not_burglary_samples/"  
not_burglary_output_dir = "not_burglary_samples"  

os.makedirs(not_burglary_output_dir, exist_ok=True)

video_files = glob.glob(os.path.join(not_burglary_samples_dir, "*.mp4"))  
print(f'Number of Videos: {len(video_files)}')

for video_file in video_files:
    frames_file = process_video(video_file, not_burglary_output_dir)
    # display_random_detected_frames(video_file, frames_file, num_frames_to_show=5)
    # break
#---------------------------------------------------------------------------------------------------------------